In [ ]:
! pip install torch==2.0.1 transformers peft accelerate trl bitsandbytes optimum auto-gptq

In [ ]:
!pip install -U datasets

### Import Libraries

In [1]:
import torch
from datasets import Dataset, load_dataset
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoTokenizer, TrainingArguments, AutoModelForCausalLM, GPTQConfig
from trl import DPOTrainer

2024-02-21 04:07:48.264220: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 04:07:48.264276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 04:07:48.265915: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Login to Huggingface

In [2]:
from huggingface_hub import notebook_login
notebook_login()

### Load Dataset

In [3]:
dataset = load_dataset(
        "HuggingFaceH4/ultrafeedback_binarized",
        split = "test_prefs",
        use_auth_token=True
)

original_columns = dataset.column_names

/opt/conda/lib/python3.10/site-packages/datasets/load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train_prefs split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating train_sft split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61135 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

### Prepare Dataset

In [4]:
def return_prompt_and_responses(samples):
    return {
        "prompt": [prompt for prompt in samples["prompt"]],
        "chosen": samples["chosen"],
        "rejected": samples["rejected"],
    }

In [5]:
train_dataset = dataset.map(
        return_prompt_and_responses,
        batched=True,
        remove_columns=original_columns,
    )

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
train_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 2000
})

In [7]:
train_df = train_dataset.to_pandas()
train_df

,prompt,chosen,rejected
0,"In this task, you are given a second sentence....","[{'content': 'In this task, you are given a se...","[{'content': 'In this task, you are given a se..."
1,The floor of a rectangular room is 19 m long a...,[{'content': 'The floor of a rectangular room ...,[{'content': 'The floor of a rectangular room ...
2,"Definition: In this task, you are given an abs...","[{'content': 'Definition: In this task, you ar...","[{'content': 'Definition: In this task, you ar..."
3,Evaluate the extent to which web usability is ...,[{'content': 'Evaluate the extent to which web...,[{'content': 'Evaluate the extent to which web...
4,A text is given in Bengali. Translate it from ...,[{'content': 'A text is given in Bengali. Tran...,[{'content': 'A text is given in Bengali. Tran...
...,...,...,...
1995,can you give me an overview of my mri medical ...,[{'content': 'can you give me an overview of m...,[{'content': 'can you give me an overview of m...
1996,"QUESTION: Can we conclude from ""Two men hold b...","[{'content': 'QUESTION: Can we conclude from ""...","[{'content': 'QUESTION: Can we conclude from ""..."
1997,Construct lyrics in the style of The Proclaime...,[{'content': 'Construct lyrics in the style of...,[{'content': 'Construct lyrics in the style of...
1998,"Detailed Instructions: In this task, you will ...",[{'content': 'Detailed Instructions: In this t...,[{'content': 'Detailed Instructions: In this t...


In [8]:
train_df["chosen"] = train_df["chosen"].apply(lambda x: x[1]["content"])
train_df["rejected"] = train_df["rejected"].apply(lambda x: x[1]["content"])

In [9]:
train_df = train_df.dropna()
val_df = train_df.sample(10)

In [10]:
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)

In [11]:
train_data

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 2000
})

### Load Tokenizer

In [12]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/OpenHermes-2-Mistral-7B-GPTQ")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_toke

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Load Model

In [13]:
model = AutoModelForCausalLM.from_pretrained("TheBloke/OpenHermes-2-Mistral-7B-GPTQ", torch_dtype=torch.float16, low_cpu_mem_usage=True, quantization_config=GPTQConfig(bits=4, disable_exllama=True))

model_ref = AutoModelForCausalLM.from_pretrained("TheBloke/OpenHermes-2-Mistral-7B-GPTQ", torch_dtype=torch.float16, low_cpu_mem_usage=True, quantization_config=GPTQConfig(bits=4, disable_exllama=True))

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


### Define Peft Config

In [14]:
peft_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=["q_proj", "v_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )
peft_config.inference_mode = False

### Load peft config to model

In [15]:
model = prepare_model_for_kbit_training(model)
model.config.use_cache=False
model.gradient_checkpointing_enable()
model.config.pretraining_tp=1
model = get_peft_model(model, peft_config)

### Define Training Arguments

In [16]:
training_args = TrainingArguments(
        per_device_train_batch_size=1,
        max_steps=50,
        remove_unused_columns=False,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        evaluation_strategy="steps",
        logging_first_step=True,
        logging_steps=10,
        output_dir="openhermes-mistral-dpo-gptq",
        optim="paged_adamw_32bit",
        warmup_steps=2,
        fp16=True,
        push_to_hub=True
    )

### Define DPO trainer

In [17]:
dpo_trainer = DPOTrainer(
        model,
        model_ref,
        args=training_args,
        beta=0.1,
        train_dataset=train_data,
        eval_dataset=val_data,
        tokenizer=tokenizer,
        max_length=512,
        max_target_length=256,
        max_prompt_length=256
    )

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### Start Training

In [18]:
dpo_trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
10,0.683900,0.641808,0.194121,0.044370,0.562500,0.149751,-447.611816,-594.149841,-1.846663,-1.739873
20,0.703800,0.628574,0.242755,0.026516,0.562500,0.216240,-447.790375,-593.663513,-1.846848,-1.750949
30,0.638800,0.621755,0.229426,-0.044639,0.625000,0.274066,-448.501923,-593.796814,-1.840106,-1.751867
40,0.648000,0.621798,0.128361,-0.129863,0.625000,0.258224,-449.354156,-594.807495,-1.833233,-1.748695
50,0.673000,0.628179,0.064839,-0.168207,0.562500,0.233046,-449.737579,-595.442688,-1.827502,-1.746451


TrainOutput(global_step=50, training_loss=0.6696784782409668, metrics={'train_runtime': 700.8183, 'train_samples_per_second': 0.071, 'train_steps_per_second': 0.071, 'total_flos': 0.0, 'train_loss': 0.6696784782409668, 'epoch': 0.03})

### Save model

In [20]:
dpo_trainer.save_model("./mistral_DPO_model")

training_args.bin:   0%|          | 0.00/4.28k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

events.out.tfevents.1708488650.9c270e1257ac.206.0:   0%|          | 0.00/12.8k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

### Push model to Hub

In [24]:
dpo_trainer.push_to_hub("openhermes-mistral-dpo-gptq")


CommitInfo(commit_url='https://huggingface.co/likhith231/openhermes-mistral-dpo-gptq/commit/4323e68bbf2b75cbaad33560673000d61295205d', commit_message='openhermes-mistral-dpo-gptq', commit_description='', oid='4323e68bbf2b75cbaad33560673000d61295205d', pr_url=None, pr_revision=None, pr_num=None)

### Inference

In [25]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("likhith231/openhermes-mistral-dpo-gptq")

inputs = tokenizer("""I have dropped my phone in water. Now it is not working what should I do now?""", return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "likhith231/openhermes-mistral-dpo-gptq",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/630 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [26]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)

I have dropped my phone in water. Now it is not working what should I do now?

If you have dropped your phone in water, the first thing you should do is to turn it off immediately. If it is still on, turn it off. Then remove the battery if possible. If the battery is not removable, then leave the phone off.

Next, you should try to dry the phone as much as possible. You can use a hair dryer or a fan to dry the phone. You can also use uncooked rice or silica gel packets to absorb the moisture.

After the phone has dried, you can try to turn it on. If it does not turn on, you can try to charge it. If it still does not turn on, then you may need to take it to a professional for repair.

If you have dropped your phone in water and it is not working, you should try to dry it as soon as possible. If it still does not work after drying, you may need to take it to a professional for repair.
183.3249475955963
